
Example code of a simple CNN network training on MNIST dataset.
The code is intended to show how to create a CNN network as well
as how to initialize loss, optimizer, etc. in a simple way to get
training to work with function that checks accuracy as well.


In [17]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset


* Stride denotes how many steps we are moving in each steps in convolution.By default it is one.
* Padding is a process of adding zeros to the input matrix symmetrically. 
* Kernel size is the size of filter used in convolutions

In [18]:
class CNN(nn.Module):
    def __init__(self,in_channels = 1,num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,out_channels = 8,kernel_size = (3,3),stride = (1,1),padding = (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (2,2),stride = (2,2))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16,kernel_size = (3,3),stride = (1,1),padding = (1,1))
        self.fc1 = nn.Linear(16*7*7 , num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        
        return x

In [19]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [20]:
#Hyperparameter
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10


In [21]:
!mkdir dataset
!ls

mkdir: cannot create directory ‘dataset’: File exists
__notebook_source__.ipynb  dataset


In [22]:
!pwd

/kaggle/working


In [23]:
#Load data
train_dataset = datasets.MNIST(root = '/kaggle/working/dataset/',train = True ,transform =transforms.ToTensor(),download = True)
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle = True)


In [24]:
test_dataset = datasets.MNIST(root = '/kaggle/working/dataset/',train = False,transform = transforms.ToTensor(),download = True)
test_loader = DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle = True)


In [25]:
# Initialize network

model = CNN().to(device)

In [26]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)


In [27]:

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()



In [28]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Cehcking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
    model.train()

In [29]:

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59571 / 60000 with accuracy 99.28
Cehcking accuracy on test data
Got 9884 / 10000 with accuracy 98.84
